In [1]:
# Extração dos dados

import pandas as pd

df = pd.read_csv('uber_data.csv')

df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [2]:
# Convertendo as datas

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['tpep_dropoff_datetime'] = pd.to_datetime(df['tpep_dropoff_datetime'])


In [3]:
# Tirando valores duplicados

df = df.drop_duplicates().reset_index(drop=True)

In [4]:
# Criando a dim_tempo

dim_tempo = df[['tpep_pickup_datetime','tpep_dropoff_datetime']].reset_index(drop=True)

dim_tempo['pick_day'] = dim_tempo['tpep_pickup_datetime'].dt.day
dim_tempo['pick_month'] = dim_tempo['tpep_pickup_datetime'].dt.month
dim_tempo['pick_year'] = dim_tempo['tpep_pickup_datetime'].dt.year
dim_tempo['pick_weekday'] = dim_tempo['tpep_pickup_datetime'].dt.weekday
dim_tempo['pick_hour'] = dim_tempo['tpep_pickup_datetime'].dt.hour

dim_tempo['drop_day'] = dim_tempo['tpep_dropoff_datetime'].dt.day
dim_tempo['drop_month'] = dim_tempo['tpep_dropoff_datetime'].dt.month
dim_tempo['drop_year'] = dim_tempo['tpep_dropoff_datetime'].dt.year
dim_tempo['drop_weekday'] = dim_tempo['tpep_dropoff_datetime'].dt.weekday
dim_tempo['drop_hour'] = dim_tempo['tpep_dropoff_datetime'].dt.hour

dim_tempo['datetime_id'] = dim_tempo.index

dim_tempo = dim_tempo[['datetime_id', 'tpep_pickup_datetime', 'pick_hour', 'pick_day', 'pick_month', 'pick_year', 'pick_weekday',
                             'tpep_dropoff_datetime', 'drop_hour', 'drop_day', 'drop_month', 'drop_year', 'drop_weekday']]

dim_tempo

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:07:55,0,1,3,2016,1
1,1,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:11:06,0,1,3,2016,1
2,2,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:31:06,0,1,3,2016,1
3,3,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
4,4,2016-03-01 00:00:00,0,1,3,2016,1,2016-03-01 00:00:00,0,1,3,2016,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,99995,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:22:15,6,1,3,2016,1
99996,99996,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:32:41,6,1,3,2016,1
99997,99997,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:37:23,6,1,3,2016,1
99998,99998,2016-03-01 06:17:10,6,1,3,2016,1,2016-03-01 06:22:09,6,1,3,2016,1


In [5]:
# Criando a dim_tipo_taxa

tipo_taxa = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

dim_tipo_taxa = pd.DataFrame(list(tipo_taxa.items()), columns=['RatecodeID', 'rate_description'])

dim_tipo_taxa

,RatecodeID,rate_description
0,1,Standard rate
1,2,JFK
2,3,Newark
3,4,Nassau or Westchester
4,5,Negotiated fare
5,6,Group ride


In [6]:
# Criando a dim_tipo_pagamento

tipo_pagamento = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}

dim_tipo_pagamento = pd.DataFrame(list(tipo_pagamento.items()), columns=['payment_type', 'payment_description'])

dim_tipo_pagamento

,payment_type,payment_description
0,1,Credit card
1,2,Cash
2,3,No charge
3,4,Dispute
4,5,Unknown
5,6,Voided trip


In [7]:
# Criando fato_corridas

fato_corridas = df.merge(dim_tipo_taxa, on = 'RatecodeID') \
                .merge(dim_tipo_pagamento, on = 'payment_type') \
                .merge(dim_tempo, on = ['tpep_pickup_datetime', 'tpep_dropoff_datetime']) \
                [['VendorID', 'datetime_id', 'RatecodeID', 'payment_type', 'passenger_count', 'trip_distance', 'pickup_longitude', 'pickup_latitude', 'store_and_fwd_flag', 'dropoff_longitude', 'dropoff_latitude', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
               'improvement_surcharge', 'total_amount']]

fato_corridas

,VendorID,datetime_id,RatecodeID,payment_type,passenger_count,trip_distance,pickup_longitude,pickup_latitude,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,0,1,1,1,2.50,-73.976746,40.765152,N,-74.004265,40.746128,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,1,1,1,1,2.90,-73.983482,40.767925,N,-74.005943,40.733166,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2,1,1,2,19.98,-73.782021,40.644810,N,-73.974541,40.675770,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,3,1,1,3,10.78,-73.863419,40.769814,N,-73.969650,40.757767,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,4,1,1,3,10.78,-73.863419,40.769814,N,-73.969650,40.757767,31.5,0.0,0.5,3.78,5.54,0.3,41.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100297,1,92805,1,4,1,0.00,-73.950027,40.750813,N,-73.950020,40.750816,2.5,0.0,0.5,0.00,0.00,0.3,3.30
100298,1,93586,1,4,1,1.80,0.000000,0.000000,N,-73.981987,40.751598,9.0,0.0,0.5,0.00,0.00,0.3,9.80
100299,1,88859,3,4,1,19.00,-73.973282,40.755424,N,-74.183662,40.700249,70.5,0.5,0.0,0.00,10.50,0.3,81.80
100300,1,66038,5,4,1,0.00,-73.832832,41.059826,N,-73.832832,41.059826,170.0,0.0,0.0,0.00,0.00,0.3,170.30
